In [ ]:
# --- Cell 0: Interactive File Setup ---
%matplotlib widget

from pathlib import Path

import ipywidgets as widgets
from ipyfilechooser import FileChooser

import opym

print("1. Select your base OME-TIF file (e.g., ...Pos0.ome.tif).")
print("2. Select the Timepoint (T) you want to use for the MIP.")
print("3. Run Cell 1 to calculate the MIP.")

# --- Create the file chooser ---
file_chooser = FileChooser(
    path="/mmfs2/scratch/SDSMT.LOCAL/bscott/DataUpload",
    filter_pattern="*.ome.tif",
    title="<b>Select Base OME-TIF File:</b>",
)

# --- Slider for Timepoint selection ---
t_slider = widgets.IntSlider(
    description="Timepoint (T):",
    min=0,
    max=199,  # Will be updated by Cell 1
    step=1,
    value=0,
)

display(file_chooser, t_slider)

In [ ]:
# --- Cell 1: Create the MIPs ---
# This cell now calls the `create_mip` function from opym

try:
    file_to_inspect = Path(file_chooser.value)
    t_index = t_slider.value

    # Call the refactored function
    mip_data, vmin, vmax, lazy_data, t_max = opym.create_mip(file_to_inspect, t_index)

    # Update the slider in Cell 0 with the correct max T
    t_slider.max = t_max
    t_slider.value = t_index

    print("\nYou can now run Cell 2 to select ROIs.")

except FileNotFoundError:
    print("❌ ERROR: File not found. Please select a file in Cell 0 and re-run.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 2: Select ROIs Interactively ---
# This cell uses the new ROISelector class from opym

try:
    # This will display the plot and wait for you to draw two ROIs
    selector = opym.interactive_roi_selector(mip_data, vmin, vmax)
except NameError:
    print("❌ ERROR: 'mip_data' not found. Please run Cell 1 first.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 3: Get ROIs and Auto-Align ---
# This cell gets the ROIs and calls the new validation/alignment function

try:
    # This call will block until two ROIs are drawn in the plot from Cell 2
    unaligned_rois = selector.get_rois()

    # --- NEW: Call the new processing function from opym ---
    # This function handles validation, alignment, and None assignment
    top_roi, bottom_roi = opym.process_rois_from_selector(
        mip_data,
        unaligned_rois,
        valid_threshold=1.0,
    )
    # --- END NEW ---

    if top_roi or bottom_roi:
        print("\nRun Cell 4 to visualize.")
    else:
        print("  Re-run Cell 2 to try again.")

except NameError:
    print(
        "❌ ERROR: 'selector' or 'mip_data' not found. Please run Cells 1 and 2 first."
    )
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 4: Visualize Final Alignment ---
# This cell calls the new visualization function from opym

from typing import cast

try:
    if top_roi is None and bottom_roi is None:
        print("❌ ERROR: No valid ROIs found. Please re-run Cell 2 and 3.")
    elif top_roi is None:
        print("ℹ️ Only Bottom ROI is valid. Skipping visualization.")
        # Cast to satisfy Pylance: we know bottom_roi is not None here
        valid_bottom = cast(tuple[slice, slice], bottom_roi)
        opym.visualize_alignment(mip_data, valid_bottom, valid_bottom, vmin, vmax)
    elif bottom_roi is None:
        print("ℹ️ Only Top ROI is valid. Skipping visualization.")
        # Cast to satisfy Pylance: we know top_roi is not None here
        valid_top = cast(tuple[slice, slice], top_roi)
        opym.visualize_alignment(mip_data, valid_top, valid_top, vmin, vmax)
    else:
        # Both are valid, show alignment
        opym.visualize_alignment(mip_data, top_roi, bottom_roi, vmin, vmax)

    print("\nRun Cell 5 to save these ROIs to your log file.")

except NameError:
    print("❌ ERROR: Required variables not found. Please run Cells 1, 2, and 3 first.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 5: Save ROIs to Log File ---
# This cell uses the save_rois_to_log function from your package

try:
    # This is the central log file used by the CLI, as per the README.
    cli_log_file = Path("opm_roi_log.json")

    if top_roi is None and bottom_roi is None:
        print("❌ ERROR: No valid ROIs to save. Please re-run Cells 2 & 3.")
    else:
        opym.save_rois_to_log(
            log_file=cli_log_file,
            base_file=file_to_inspect,
            top_roi=top_roi,  # Can be None
            bottom_roi=bottom_roi,  # Can be None
        )
        print(f"✅ ROIs saved to {cli_log_file.name}")
        print("You can now use these ROIs for batch processing via the CLI,")
        print("or run Cell 6 to process this file immediately.")

except NameError:
    print("❌ ERROR: Required variables not found. Please run all previous cells.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 6: Submit Cropping Job (Dynamic Channels & Remote Execution) ---

import importlib
import sys
import re
from pathlib import Path
from typing import cast
import tifffile

import ipywidgets as widgets
from IPython.display import display

# --- 1. Import the opym package ---
try:
    import opym
    import opym.petakit  # Import the new petakit module
    from opym.utils import OutputFormat

    # Reload modules to pick up any changes
    importlib.reload(opym.utils)
    importlib.reload(opym.metadata)
    importlib.reload(opym.petakit) 
    importlib.reload(opym)
    print("Successfully re-loaded opym package.")

    # --- 2. Detect Channels Dynamically ---
    try:
        # Check if file_to_inspect is defined (from previous cells)
        if 'file_to_inspect' not in globals():
            raise NameError("file_to_inspect is not defined. Run previous cells.")

        with tifffile.TiffFile(file_to_inspect) as tif:
            if hasattr(tif, "series") and len(tif.series) > 0:
                 shape = tif.series[0].shape
                 ndim = len(shape)
                 # Heuristic for OME-TIFF dims
                 if ndim == 5: n_channels_in = shape[2]
                 elif ndim == 4: n_channels_in = shape[1]
                 else:
                     summary = tif.micromanager_metadata.get('Summary', {}) # type: ignore
                     n_channels_in = summary.get('Channels', 2)
            else:
                 n_channels_in = 4 # Default fallback
    except Exception as e:
        print(f"Warning: Could not auto-detect channels ({e}). Defaulting to 4 inputs.")
        n_channels_in = 4

    # Logic: 4 Input Channels (Raw) -> 8 Output Channels (Split)
    n_excitations = max(1, n_channels_in // 2) 
    print(f"Detected {n_channels_in} Input Channels -> {n_excitations} Excitation group(s).")

    # --- 3. Create Widgets ---
    output_format_widget = widgets.RadioButtons(
        options=[(f.value, f) for f in OutputFormat],
        description="Output Format:",
        value=OutputFormat.TIFF_SERIES,
        disabled=False,
    )

    rotate_widget = widgets.Checkbox(
        value=True, description="Rotate 90° (CCW)", disabled=False, indent=False
    )

    # --- Dynamic Checkbox Generation ---
    channel_checks = {}
    ui_rows = []
    
    ui_rows.append(widgets.Label(f"Channels to Output ({n_channels_in*2} possible outputs):"))

    for exc in range(n_excitations):
        base_id = exc * 4
        
        lbl = widgets.Label(f"--- Group {exc+1} (Camera 1 & Camera 2) ---")
        ui_rows.append(lbl)
        
        # Raw Ch N -> Outputs 2N, 2N+1
        c_bot0 = widgets.Checkbox(value=True, description=f"C{base_id} (Bot, Cam 1)")
        c_top0 = widgets.Checkbox(value=True, description=f"C{base_id+1} (Top, Cam 1)")
        c_top1 = widgets.Checkbox(value=True, description=f"C{base_id+2} (Top, Cam 2)")
        c_bot1 = widgets.Checkbox(value=True, description=f"C{base_id+3} (Bot, Cam 2)")
        
        channel_checks[base_id]   = c_bot0
        channel_checks[base_id+1] = c_top0
        channel_checks[base_id+2] = c_top1
        channel_checks[base_id+3] = c_bot1
        
        ui_rows.append(widgets.HBox([c_bot0, c_top0]))
        ui_rows.append(widgets.HBox([c_top1, c_bot1]))

    channel_box = widgets.VBox(ui_rows)

    run_button = widgets.Button(
        description="Submit to PetaKit Queue", 
        button_style="primary", 
        icon="server"
    )
    
    status_label = widgets.Label(value="Ready to submit.")
    run_output = widgets.Output()

    display(
        widgets.VBox(
            [
                output_format_widget,
                rotate_widget,
                channel_box,
                widgets.HBox([run_button, status_label]), 
                run_output,
            ]
        )
    )

    # --- 4. Define global vars to store paths ---
    processing_output_dir: Path | None = None
    processing_format: OutputFormat | None = None

    # --- 5. Define the function to run on button click ---
    def on_run_button_clicked(b):
        global processing_output_dir, processing_format

        with run_output:
            run_output.clear_output()

            try:
                # --- Get selected channels ---
                channels_to_output = []
                for cid, checkbox in channel_checks.items():
                    if checkbox.value:
                        channels_to_output.append(cid)
                channels_to_output.sort()
                
                if not channels_to_output:
                    print("❌ ERROR: No output channels selected.")
                    return

                # --- Validate ROIs ---
                # Cam 1 (Odd Raw) -> Outputs 0,1, 4,5 ...
                # Cam 2 (Even Raw) -> Outputs 2,3, 6,7 ...
                # Actually, simply: If ANY output selected, we likely need both ROIs
                # unless you specifically selected only one half.
                if top_roi is None:
                    print("❌ ERROR: Top ROI is missing.")
                    return
                if bottom_roi is None:
                    print("❌ ERROR: Bottom ROI is missing.")
                    return

                # --- Run Job (REMOTE SUBMISSION) ---
                processing_format = cast(OutputFormat, output_format_widget.value)
                rotate_90_value = rotate_widget.value

                print("🚀 Submitting job to Petakit Queue...")
                
                job_path = opym.petakit.submit_remote_crop_job(
                    base_file=file_to_inspect,
                    top_roi=top_roi,
                    bottom_roi=bottom_roi,
                    channels=channels_to_output,
                    output_format=processing_format.value,
                    rotate=rotate_90_value
                )
                
                print(f"✅ Job Ticket Created: {job_path.name}")
                print("   The job is running in the background.")
                
                # --- UPDATE OUTPUT PATH CORRECTLY ---
                # We stripped the extension in the cropper, so we replicate that here
                # e.g. 'Data.ome.tif' -> 'Data'
                base_name = file_to_inspect.name
                if base_name.lower().endswith(".ome.tif"):
                    clean_name = base_name[:-8]
                elif base_name.lower().endswith(".tif"):
                    clean_name = base_name[:-4]
                else:
                    clean_name = file_to_inspect.stem

                processing_output_dir = file_to_inspect.parent / clean_name
                print(f"   Target Output Dir: {processing_output_dir.name}")
                
                # --- Async Background Monitor ---
                status_label.value = "⏳ Initializing..."
                opym.petakit.monitor_job_background(job_path, status_label)

            except NameError as e:
                print(f"❌ ERROR: Variable missing: {e}", file=sys.stderr)
            except Exception as e:
                import traceback
                traceback.print_exc()

    # --- 6. Link the button ---
    run_button.on_click(on_run_button_clicked)

except ImportError:
    print("❌ ERROR: Could not import opym package.", file=sys.stderr)

In [ ]:
# --- Cell 7: Run PyPetaKit5D Processing (Async Background) ---

import importlib
import re
import sys
from pathlib import Path

import ipywidgets as widgets
from IPython.display import display

import opym
import opym.metadata
import opym.petakit

# --- Reload modules ---
try:
    importlib.reload(opym.metadata)
    importlib.reload(opym.petakit)
    importlib.reload(opym)
except Exception:
    pass

# --- Create Widgets ---
petakit_button = widgets.Button(
    description="Submit PetaKit Job (Deskew/Rotate)", 
    button_style="success",
    icon="rocket"
)
# A label to show the background thread status
status_label = widgets.Label(value="Ready to submit.")
petakit_output = widgets.Output()

display(widgets.VBox([
    widgets.HBox([petakit_button, status_label]),
    petakit_output
]))

# --- Define the function to run on button click ---
def on_petakit_button_clicked(b):
    with petakit_output:
        petakit_output.clear_output()

        try:
            # 1. Validation (Optional check, depends on previous cells)
            # Ensure processing_output_dir is defined in your notebook
            if 'processing_output_dir' not in globals() or processing_output_dir is None:
                print("❌ ERROR: 'processing_output_dir' not set. Please run Cell 6 first.")
                return

            # 2. Metadata Parsing (Z-Step)
            print("--- Parsing Metadata ---")
            
            # Helper to find base name
            base_name = None
            
            # Strategy A: Check for existing log file
            log_file = next(processing_output_dir.glob("*_processing_log.json"), None)
            if log_file:
                base_name = log_file.stem.replace("_processing_log", "")
            else:
                # Strategy B: Scan for TIFFs (Updated for C00_T0000 format)
                # Matches: anything_C(digits)_T(digits).tif
                pattern = re.compile(r"^(.*?)_C\d+_T\d+\.tif$")
                
                first_file = next(processing_output_dir.glob("*.tif"), None)
                if first_file:
                    match = pattern.match(first_file.name)
                    if match:
                        base_name = match.group(1)

            z_step = 1.0
            if base_name:
                # Try finding metadata in the output folder (copied by cropper) or parent
                metadata_candidates = [
                    processing_output_dir / (base_name + "_metadata.txt"), # Copied by cropper
                    processing_output_dir.parent / (base_name + "_metadata.txt"), # Original location
                    processing_output_dir / "AcqSettings.txt" # Common fallback
                ]
                
                meta_found = False
                for meta_file in metadata_candidates:
                    if meta_file.exists():
                        z_step = opym.metadata.parse_z_step(meta_file, default_z_step=1.0)
                        print(f"   Detected Z-step: {z_step} µm (from {meta_file.name})")
                        meta_found = True
                        break
                
                if not meta_found:
                    print(f"⚠️ Metadata not found. Defaulting to 1.0 µm.")
            else:
                print("⚠️ Could not determine base name from files. Defaulting to 1.0 µm.")

            # 3. Submit Async Job (Updated Signature)
            print(f"\n🚀 Submitting Job for: {processing_output_dir.name}")
            
            job_path = opym.petakit.submit_remote_deskew_job(
                input_target=processing_output_dir,  # <--- UPDATED (was input_dir)
                z_step_um=z_step,
                xy_pixel_size=0.136,
                sheet_angle_deg=31.8,
                deskew=True,
                rotate=True
            )
            
            print(f"✅ Ticket Created: {job_path.name}")
            print("   The job is now running in the background.")
            print("   You can continue working in other cells!")

            # 4. Start Background Monitor
            status_label.value = "⏳ Initializing..."
            opym.petakit.monitor_job_background(job_path, status_label)

        except FileNotFoundError as e:
            print("\n❌ ERROR: Could not find required directories.")
            print(f"   Details: {e}")
        except Exception as e:
            # Print full trace for debugging
            import traceback
            traceback.print_exc()

petakit_button.on_click(on_petakit_button_clicked)

In [ ]:
# --- Cell 8: Load Final Processed Data ---
# This cell loads the final deskewed + rotated TIFF series
# from the 'DSR' directory created in Cell 7.

import sys
import importlib
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display

# Import your package
import opym 
# Ensure dataloader is refreshed
importlib.reload(opym.dataloader)

# --- SAFEGUARD: Initialize globals if they don't exist ---
# This fixes the Pylance "UnboundVariable" errors
if "processing_output_dir" not in globals():
    processing_output_dir = None
if "viewer_params" not in globals():
    viewer_params = None

# --- Create Widgets ---
load_dsr_button = widgets.Button(
    description="Load Final DSR Data", 
    button_style="primary",
    icon="folder-open"
)
load_dsr_output = widgets.Output()

# --- Define the function to run on button click ---
def on_load_dsr_button_clicked(b):
    global viewer_params, processing_output_dir

    with load_dsr_output:
        load_dsr_output.clear_output()

        try:
            # 1. Resolve the Target Directory
            target_dsr = None
            
            # Scenario A: We have the variable from previous cells
            if processing_output_dir is not None:
                candidate = processing_output_dir / "DSR"
                if candidate.exists():
                    target_dsr = candidate
            
            # Scenario B: Variable missing, try to auto-detect
            if target_dsr is None:
                print("⚠️ 'processing_output_dir' not set. Searching for recent 'DSR' folders...")
                # Look for any folder ending in 'DSR' in the current directory
                found_dsrs = sorted(list(Path(".").glob("*/DSR")), key=lambda p: p.stat().st_mtime, reverse=True)
                
                if found_dsrs:
                    target_dsr = found_dsrs[0] # Pick the most recently modified one
                    print(f"   -> Found auto-detected folder: {target_dsr}")
                else:
                    print("❌ ERROR: Could not auto-detect any 'DSR' folders.")
                    print("   Please run Cell 6 & 7 first, or ensure your output folder exists.")
                    return

            print(f"--- Loading data from: {target_dsr} ---")

            # 2. Call the data loader
            (
                get_stack,
                t_min,
                t_max,
                c_min,
                c_max,
                z_max,
                y,
                x,
                base_name,
            ) = opym.load_tiff_series(target_dsr)

            # 3. Store parameters in the global variable
            viewer_params = (get_stack, t_max, z_max, c_max, y, x)

            print(
                f"✅ Data loaded successfully.\n"
                f"   Shape: T={t_min}-{t_max}, Z={z_max + 1}, C={c_min}-{c_max}, Y={y}, X={x}\n"
                f"   Base name: {base_name}"
            )
            print("\nUsage: Run the viewer cell below to visualize.")

        except FileNotFoundError as e:
            print("\n❌ ERROR: Could not load data.")
            print(f"  Details: {e}")
            print("  Check if the 'DSR' folder is empty or contains incompatible files.")
        except Exception as e:
            import traceback
            traceback.print_exc()
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)


# --- Link button and display UI ---
load_dsr_button.on_click(on_load_dsr_button_clicked)
display(widgets.VBox([load_dsr_button, load_dsr_output]))

In [ ]:
# --- Cell 9: Launch the single-channel viewer ---
try:
    if viewer_params is None:
        raise NameError
    opym.single_channel_viewer(*viewer_params)
except NameError:
    print("❌ ERROR: 'viewer_params' not found. Please run Cell 8 to load data first.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

In [ ]:
# --- Cell 10: Launch the composite viewer ---
try:
    if viewer_params is None:
        raise NameError
    opym.composite_viewer(*viewer_params)
except NameError:
    print("❌ ERROR: 'viewer_params' not found. Please run Cell 8 to load data first.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

In [ ]:
# --- Cell 11: Batch Process Remaining Cells (Crop -> Deskew Chain) ---
import time
import sys
from pathlib import Path
import opym.petakit
from opym.utils import OutputFormat

# --- 1. Validate Prerequisites ---
# Ensure we have the settings from the interactive cells
required_vars = ['channel_checks', 'output_format_widget', 'rotate_widget', 'top_roi', 'bottom_roi', 'file_to_inspect']
missing = [v for v in required_vars if v not in globals()]

if missing:
    print("❌ ERROR: Missing required interactive variables.")
    print(f"   Missing: {missing}")
    print("   Please run Cell 6 (and ensure ROIs are drawn) before running this batch cell.")
    raise RuntimeError("Prerequisites missing")

# --- 2. Extract Settings from Widgets ---
# Get Channels
channels_to_output = [cid for cid, checkbox in channel_checks.items() if checkbox.value]
channels_to_output.sort()

if not channels_to_output:
    print("❌ ERROR: No output channels selected in Cell 6 widgets.")
    raise ValueError("No channels selected")

# Get Format/Rotation
processing_format = output_format_widget.value
rotate_90_value = rotate_widget.value

print(f"⚙️  Batch Settings:")
print(f"   • Channels: {channels_to_output}")
print(f"   • Format:   {processing_format}")
print(f"   • Rotate:   {rotate_90_value}")
print(f"   • Top ROI:  {top_roi}")
print(f"   • Bot ROI:  {bottom_roi}")

# --- 3. Scan for Files ---
# Go up two levels from the inspected file to find the dataset root
data_root = file_to_inspect.parent.parent 
print(f"\n📂 Scanning directory: {data_root}")

# Find all matching .ome.tif files
all_files = sorted(list(data_root.glob("**/*_MMStack_Pos0.ome.tif")))

# Filter out the one we already processed interactively
files_to_process = [f for f in all_files if f.resolve() != file_to_inspect.resolve()]

if not files_to_process:
    print("⚠️ No other files found to process.")
else:
    print(f"🔎 Found {len(all_files)} total files.")
    print(f"🚀 Queueing {len(files_to_process)} files for batch processing...")

# --- 4. Loop and Submit (Chain: Crop -> Deskew) ---
submitted_jobs = []

for i, file_path in enumerate(files_to_process):
    print(f"\n[{i+1}/{len(files_to_process)}] Processing: {file_path.name}")
    
    try:
        # A. Submit CROP Job
        crop_ticket = opym.petakit.submit_remote_crop_job(
            base_file=file_path,
            top_roi=top_roi,
            bottom_roi=bottom_roi,
            channels=channels_to_output,
            output_format=processing_format,
            rotate=rotate_90_value
        )
        
        # B. Calculate where the output will be
        # (Matches logic in petakit.py / run_bigtiff_cropper.m)
        base_name = file_path.name
        if base_name.lower().endswith(".ome.tif"):
            clean_name = base_name[:-8]
        elif base_name.lower().endswith(".tif"):
            clean_name = base_name[:-4]
        else:
            clean_name = file_path.stem
            
        target_dir = file_path.parent / clean_name
        
        # C. Submit DESKEW Job (Chained)
        # Note: We don't wait for Crop to finish; we just submit both tickets.
        # The PetaKit server processes them in FIFO order, so Deskew will pick up
        # after Crop finishes (assuming single-threaded consumer or specific logic).
        # *Critically*, if your server runs parallel jobs, Deskew might fail if Crop isn't done.
        # SAFEGUARD: We will assume standard PetaKit queue behavior.
        
        # Detect Z-step from metadata if possible, else default
        # (Simplified for batch: assumes all files in folder share settings)
        z_step = 1.0 
        meta_file = file_path.parent / "AcqSettings.txt"
        if meta_file.exists():
             # Quick parse if opym.metadata is available
             try:
                 z_step = opym.metadata.parse_z_step(meta_file, 1.0)
             except:
                 pass

        deskew_ticket = opym.petakit.submit_remote_deskew_job(
            input_target=target_dir, # Points to the future output folder
            z_step_um=z_step,
            xy_pixel_size=0.136,
            sheet_angle_deg=31.8,
            deskew=True,
            rotate=True
        )
        
        submitted_jobs.append((file_path.name, crop_ticket, deskew_ticket))
        print(f"   ✅ Submitted Crop & Deskew tickets.")

    except Exception as e:
        print(f"   ❌ Failed to submit {file_path.name}: {e}")

# --- 5. Monitor Progress ---
if submitted_jobs:
    print(f"\n⏳ Monitoring {len(submitted_jobs)*2} Jobs (Crop + Deskew)...")
    
    # Simple monitoring loop
    try:
        while True:
            all_done = True
            pending = 0
            
            # Check status of the last batch of tickets
            # (We check the Deskew tickets since they are the final step)
            for name, crop_t, deskew_t in submitted_jobs:
                base = deskew_t.parent.parent # petakit_jobs root
                if (base / "queue" / deskew_t.name).exists():
                    pending += 1
                    all_done = False
                elif (base / "queue" / crop_t.name).exists():
                     pending += 1
                     all_done = False
            
            sys.stdout.write(f"\r📊 Queue Status: {pending} jobs pending...   ")
            sys.stdout.flush()
            
            if all_done:
                break
            time.sleep(2)
            
        print("\n\n✅ All batch jobs completed!")
        
    except KeyboardInterrupt:
        print("\n🛑 Monitoring stopped (Jobs continue running in background).")